In [4]:
import pandas as pd
from sqlalchemy import create_engine
from string import digits
import numpy as np
import os
import nltk
import string
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI
# nltk.download('stopwords')

In [ ]:
# from nltk.corpus import brown
# brown.words()

In [7]:
csv1 = "./indeed_job_dataset_jds_v3.csv"
csv2 = "./indeed_job_dataset_V5_CSV .csv"
df1 = pd.read_csv(csv1, encoding = 'unicode_escape')
df2 = pd.read_csv(csv2)
df1.head()
# df2.head()

,Index_No,Job_Title,Link,Queried_Salary,Job_Type,Skill,Description,Description_and_Skill
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']","[POSITION SUMMARY, \r\r\nThe Business Analyst ...","[POSITION SUMMARY, The Business Analyst role i..."
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']","[What do we need?, You to have an amazing pers...","[What do we need?, You to have an amazing pers..."
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...","[Validate, analyze, and conduct statistical an...","[Validate, analyze, and conduct statistical an..."
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],"[Full time, Washington, DC metro area, Startin...","[Full time, Washington, DC metro area, Startin..."
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",[Assist in consultations with business partner...,[Assist in consultations with business partner...


In [8]:
df1 = df1.drop(columns=['Job_Type', 'Queried_Salary', 'Link', 'Job_Title', 'Skill', 'Description'])
df1.head()

,Index_No,Description_and_Skill
0,0,"[POSITION SUMMARY, The Business Analyst role i..."
1,1,"[What do we need?, You to have an amazing pers..."
2,2,"[Validate, analyze, and conduct statistical an..."
3,3,"[Full time, Washington, DC metro area, Startin..."
4,4,[Assist in consultations with business partner...


In [9]:
# Combine the data into a single dataset.  
df = pd.merge(df1, df2, how="left", on=["Index_No"])
df.head()

,Index_No,Description_and_Skill,Job_Title,Job_Type,Link,Queried_Salary,<80000,80000-99999,100000-119999,120000-139999,...,spss,softwaredevelopment,shellscripting,datascience,docker,mongodb,.net,projectmanagement,businessintelligence,s3
0,0,"[POSITION SUMMARY, The Business Analyst role i...",Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,"[What do we need?, You to have an amazing pers...",Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,"[Validate, analyze, and conduct statistical an...",Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,3,"[Full time, Washington, DC metro area, Startin...",Graduate Studies Program - Data Scientist,Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,[Assist in consultations with business partner...,Data Scientist I,Data Scientist,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [10]:
df3 = df[['Job_Type', 'Description_and_Skill']]
df3.head()

,Job_Type,Description_and_Skill
0,Data Scientist,"[POSITION SUMMARY, The Business Analyst role i..."
1,Data Scientist,"[What do we need?, You to have an amazing pers..."
2,Data Scientist,"[Validate, analyze, and conduct statistical an..."
3,Data Scientist,"[Full time, Washington, DC metro area, Startin..."
4,Data Scientist,[Assist in consultations with business partner...


In [11]:
# df3.to_csv('df3.csv')
# df.describe()
# df.dtypes
# len(df3['Description'])

In [12]:
df3.dropna(inplace=True)
len(df3['Description_and_Skill'])

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


5676

In [13]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_words

In [14]:
df['Description_and_Skill'].head().apply(process_text)

0    [POSITION, SUMMARY, Business, Analyst, role, p...
1    [need, amazing, personality, communication, st...
2    [Validate, analyze, conduct, statistical, anal...
3    [Full, time, Washington, DC, metro, area, Star...
4    [Assist, consultations, business, partners, in...
Name: Description_and_Skill, dtype: object

In [15]:
#convert collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
count_v  = CountVectorizer(analyzer=process_text)
message_bow = count_v.fit_transform(df3['Description_and_Skill'])

In [ ]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_bow, df3['Job_Type'],test_size=0.20, random_state=0)

In [ ]:
message_bow.shape

In [ ]:
#create and train the naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier =  MultinomialNB().fit(X_train, y_train)

In [ ]:
#print the prediction
print(classifier.predict(X_train))
#print values
print(y_train.values)

In [ ]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_train)
print(classification_report(y_train, prediction))
print()

In [ ]:
print('Confusion Matrix: \n', confusion_matrix(y_train,prediction))
print('Accuracy Matrix:',  accuracy_score(y_train,prediction))

In [ ]:
#print the prediction
print(classifier.predict(X_test))
#print values
print(y_test.values)


In [ ]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_test)
print(classification_report(y_test, prediction))
print()
print('Confusion Matrix: \n', confusion_matrix(y_test,prediction))
print()
print('Accuracy Matrix:',  accuracy_score(y_test,prediction))

In [ ]:
# print(X_train)

In [ ]:
example = ['data analyst is cool. i know sql, python, and tableau. I am studious and adaptable.']
input_bow = count_v.transform(example)
print(input_bow)
# classifier =  MultinomialNB().fit(X1_train, y1_train)
classifier.predict(input_bow)

In [ ]:
# class NaiveBayesClassifier(ClassifierI):